<div class="alert alert-block alert-success">
    <h1 align="center">Titanic Dataset</h1>
    <h3 align="center">Make final model and use it in Streamlit</h3>
    <h4 align="center"><a href="http://www.iran-machinelearning.ir">Soheil Tehranipour</a></h5>
</div>

In [ ]:
# !pip install sklearn xgboost

In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer , make_column_transformer
from sklearn.base import BaseEstimator, TransformerMixin
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import warnings
warnings. filterwarnings('ignore')

In [ ]:
df = pd.read_csv('train.csv')
X = df.drop('Survived', axis=1) 
y = df.Survived

In [ ]:
df.head()

In [ ]:
# The BaseEstimator just gives it the get_params and set_params methods that all Scikit-learn estimators require. 
#The TransformerMixin gives it the fit_transform method.

class PrepProcesor(BaseEstimator, TransformerMixin): 
    def fit(self, X, y=None): 
        self.ageImputer = SimpleImputer()
        self.ageImputer.fit(X[['Age']])        
        return self 
        
    def transform(self, X, y=None):
        X['Age'] = self.ageImputer.transform(X[['Age']])
        X['CabinClass'] = X['Cabin'].fillna('M').apply(lambda x: str(x).replace(" ", "")).apply(lambda x: re.sub(r'[^a-zA-Z]', '', x))
        X['CabinNumber'] = X['Cabin'].fillna('M').apply(lambda x: str(x).replace(" ", "")).apply(lambda x: re.sub(r'[^0-9]', '', x)).replace('', 0) 
        X['Embarked'] = X['Embarked'].fillna('M')
        X = X.drop(['PassengerId', 'Name', 'Ticket','Cabin'], axis=1)
        return X

In [ ]:
preproc = PrepProcesor()
numeric_pipeline = Pipeline([('Scaler', StandardScaler())]) 
categorical_pipeline = Pipeline([('OneHot', OneHotEncoder(handle_unknown='ignore'))])
transformer = ColumnTransformer([('num', numeric_pipeline, ['Pclass','Age','SibSp','Parch','Fare','CabinNumber']), ('cat', categorical_pipeline, ['Sex','Embarked','CabinClass'])])


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.10, random_state=1234) 

In [ ]:
X_train

In [ ]:
mlpipe = Pipeline([('InitialPreproc', PrepProcesor()), ('Transformer',transformer), ('xgb', XGBClassifier())])

In [ ]:
mlpipe.fit(X_train,y_train)

In [ ]:
y_hat = mlpipe.predict(X_test) 

In [ ]:
y_hat.shape

In [ ]:
y_test

In [ ]:
precision_score(y_test, y_hat) 

In [ ]:
import joblib

In [ ]:
joblib.dump(mlpipe, 'xgbpipe.joblib') 

In [ ]:
model = joblib.load('xgbpipe.joblib')

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
test.columns

In [ ]:
yhat = mlpipe.predict(test)
yhat

<img src="https://webna.ir/wp-content/uploads/2018/08/%D9%85%DA%A9%D8%AA%D8%A8-%D8%AE%D9%88%D9%86%D9%87.png" width=50% />